In [ ]:
import numpy as np
import pandas as pd 
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from nltk.stem.porter import PorterStemmer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import spacy
!python -m spacy download uk_core_news_sm
!python -m spacy download ru_core_news_sm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2022-11-29 13:16:17.973685: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.9 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('uk_core_news_sm')


In [ ]:
# source: https://github.com/Desklop/Uk_Stemmer/blob/master/uk_stemmer/uk_stemmer.py
class UkrainianStemmer():
    def __init__(self, word):
        self.word = word
        self.vowel = r'аеиоуюяіїє'  # http://uk.wikipedia.org/wiki/Голосний_звук
        self.perfectiveground = r'(ив|ивши|ившись|ыв|ывши|ывшись((?<=[ая])(в|вши|вшись)))$'
        # http://uk.wikipedia.org/wiki/Рефлексивне_дієслово
        self.reflexive = r'(с[яьи])$'
        # http://uk.wikipedia.org/wiki/Прикметник + http://wapedia.mobi/uk/Прикметник
        self.adjective = r'(ими|ій|ий|а|е|ова|ове|ів|є|їй|єє|еє|я|ім|ем|им|ім|их|іх|ою|йми|іми|у|ю|ого|ому|ої)$'
        # http://uk.wikipedia.org/wiki/Дієприкметник
        self.participle = r'(ий|ого|ому|им|ім|а|ій|у|ою|ій|і|их|йми|их)$'
        # http://uk.wikipedia.org/wiki/Дієслово
        self.verb = r'(сь|ся|ив|ать|ять|у|ю|ав|али|учи|ячи|вши|ши|е|ме|ати|яти|є)$'
        # http://uk.wikipedia.org/wiki/Іменник
        self.noun = r'(а|ев|ов|е|ями|ами|еи|и|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я|і|ові|ї|ею|єю|ою|є|еві|ем|єм|ів|їв|ю)$'
        self.rvre = r'[аеиоуюяіїє]'
        self.derivational = r'[^аеиоуюяіїє][аеиоуюяіїє]+[^аеиоуюяіїє]+[аеиоуюяіїє].*(?<=о)сть?$'
        self.RV = ''

    def ukstemmer_search_preprocess(self, word):
        word = word.lower()
        word = word.replace("'", "")
        word = word.replace("ё", "е")
        word = word.replace("ъ", "ї")
        return word

    def s(self, st, reg, to):
        orig = st
        self.RV = re.sub(reg, to, st)
        return (orig != self.RV)

    def stem_word(self):
        word = self.ukstemmer_search_preprocess(self.word)
        if not re.search('[аеиоуюяіїє]', word):
            stem = word
        else:
            p = re.search(self.rvre, word)
            start = word[0:p.span()[1]]
            self.RV = word[p.span()[1]:]

            # Step 1
            if not self.s(self.RV, self.perfectiveground, ''):

                self.s(self.RV, self.reflexive, '')
                if self.s(self.RV, self.adjective, ''):
                    self.s(self.RV, self.participle, '')
                else:
                    if not self.s(self.RV, self.verb, ''):
                        self.s(self.RV, self.noun, '')
            # Step 2
            self.s(self.RV, 'и$', '')

            # Step 3
            if re.search(self.derivational, self.RV):
                self.s(self.RV, 'ость$', '')

            # Step 4
            if self.s(self.RV, 'ь$', ''):
                self.s(self.RV, 'ейше?$', '')
                self.s(self.RV, 'нн$', u'н')

            stem = start + self.RV
        return stem


In [ ]:
# Вписати шлях до датасету (*.csv)
df = pd.read_csv('../db.csv', index_col=['id'])
print(df)
df['parsed_text'] = df.parsed_text.str.lower()
df['parsed_text'] = df.parsed_text.apply(lambda x: re.sub(r'[^\w\s]', '', x))
df

                                            parsed_text propaganda  \
id                                                                   
1147  ​​Китайська жіночка-кішка продовжує намагатись...          f   
2     1991:\nРоссия🇸🇰👍 — давайте не будем ссориться,...          t   
4     🚨Мысли вслух каждого росисянина🍾🇸🇮 сейчас слов...          t   
5     Комментарий Пескова по поводу диалога Эстонии ...          t   
6     Реакции мы решили включить, ставим лайки за си...          t   
...                                                 ...        ...   
1058  Байден молов багато зайвого, і в тому числі по...          f   
1098  ​​До речі, Дуче про превентивну ящерку правий....          f   
1113  Ми зараз знаходились в схожому моменті на люти...          f   
1122  ​​Улюблена сучка Такера та об‘єкт мастурбації ...          f   
1123  ​​Смішно спостерігати за цим псевдоправим бляд...          f   

     language_code  
id                  
1147            ua  
2              rus  
4    

,parsed_text,propaganda,language_code
id,,,
1147,китайська жіночкакішка продовжує намагатись в ...,f,ua
2,1991\nроссия давайте не будем ссориться мы ве...,t,rus
4,мысли вслух каждого росисянина сейчас словами ...,t,rus
5,комментарий пескова по поводу диалога эстонии ...,t,rus
6,реакции мы решили включить ставим лайки за сил...,t,rus
...,...,...,...
1058,байден молов багато зайвого і в тому числі пов...,f,ua
1098,до речі дуче про превентивну ящерку правий \n\...,f,ua
1113,ми зараз знаходились в схожому моменті на люти...,f,ua


In [ ]:
# df[df['parsed_text'].str.len() == 0]['parsed_text'] = None
# print(df[df['parsed_text'].str.len() == 0])
df.iloc[[80, 266]] = None
df.dropna(inplace=True)
# print(len(df))

In [ ]:
temp = list(map(word_tokenize, df.parsed_text.values))
df['parsed_text'] = temp
# lem = WordNetLemmatizer()
print(temp)
df

[['китайська', 'жіночкакішка', 'продовжує', 'намагатись', 'в', 'пропаганду'], ['1991', 'россия', 'давайте', 'не', 'будем', 'ссориться', 'мы', 'ведь', 'братья', 'украина', 'мы', 'бандеры', 'мы', 'хотим', 'майданы', 'и', 'разруху', 'ущемление', 'русских', 'россия', 'пожалуйста', 'давайте', 'сохраним', 'совок', 'украина', 'иди', 'нахй', '1999', 'украина', 'бомбит', 'югославию', 'россия', 'украина', 'пожалуйста', 'остановись', 'украина', 'иди', 'нахуй', '2004', 'украина', 'проводит', 'майданы', 'и', 'ущемление', 'русских', 'россия', 'давай', 'все', 'будет', 'мирно', 'и', 'мы', 'будем', 'дружить', 'украина', 'иди', 'нахй', '2013', 'украина', 'проводит', 'кровавые', 'незаконные', 'майданы', 'и', 'ущемляет', 'русских', 'россия', 'украина', 'остановись', 'в', 'крыму', 'вообще', 'плохо', 'нам', 'в', 'случае', 'чего', 'придется', 'помогать', 'путя', 'спасибо', 'за', 'крым', 'украина', 'иди', 'нахй', '20142022', 'украина', 'бомбит', 'донбасс', 'и', 'геноцидит', 'русских', 'вступает', 'в', 'нато',

,parsed_text,propaganda,language_code
id,,,
1147,"[китайська, жіночкакішка, продовжує, намагатис...",f,ua
2,"[1991, россия, давайте, не, будем, ссориться, ...",t,rus
4,"[мысли, вслух, каждого, росисянина, сейчас, сл...",t,rus
5,"[комментарий, пескова, по, поводу, диалога, эс...",t,rus
6,"[реакции, мы, решили, включить, ставим, лайки,...",t,rus
...,...,...,...
1058,"[байден, молов, багато, зайвого, і, в, тому, ч...",f,ua
1098,"[до, речі, дуче, про, превентивну, ящерку, пра...",f,ua
1113,"[ми, зараз, знаходились, в, схожому, моменті, ...",f,ua


In [ ]:

def stemming(parsed_text):
  stem_parsed_text = []
  for word in parsed_text:
    ukrainian_stemmer = UkrainianStemmer(word)
    stem_parsed_text.append(ukrainian_stemmer.stem_word())

  return stem_parsed_text
  
print(df.parsed_text)
df['parsed_text'] = df['parsed_text'].apply(lambda x: stemming(x))
print(df.parsed_text)


id
1147    [китайська, жіночкакішка, продовжує, намагатис...
2       [1991, россия, давайте, не, будем, ссориться, ...
4       [мысли, вслух, каждого, росисянина, сейчас, сл...
5       [комментарий, пескова, по, поводу, диалога, эс...
6       [реакции, мы, решили, включить, ставим, лайки,...
                              ...                        
1058    [байден, молов, багато, зайвого, і, в, тому, ч...
1098    [до, речі, дуче, про, превентивну, ящерку, пра...
1113    [ми, зараз, знаходились, в, схожому, моменті, ...
1122    [улюблена, сучка, такера, та, обєкт, мастурбац...
1123    [смішно, спостерігати, за, цим, псевдоправим, ...
Name: parsed_text, Length: 1135, dtype: object
id
1147    [китайськ, жіночкакішк, продовж, намаг, в, про...
2       [1991, росс, давайт, не, буд, ссорит, мы, вед,...
4       [мысли, вслух, кажд, росисянин, сейчас, слов, ...
5       [комментар, песк, по, повод, диалог, эстон, и,...
6       [реакц, мы, решил, включит, став, лайк, за, си...
                   

In [ ]:
my_vectorizer = TfidfVectorizer()

In [ ]:
df['parsed_text'] = df['parsed_text'].apply(lambda x: ' '.join(x))
print(df['parsed_text'])
def get_all_words(all_parsed_text):
  for x in all_parsed_text:
    yield x
    # for y in x:
    #   yield y

my_vectorizer.fit(get_all_words(df.parsed_text))






id
1147       китайськ жіночкакішк продовж намаг в пропаганд
2       1991 росс давайт не буд ссорит мы вед брат укр...
4       мысли вслух кажд росисянин сейчас слов админис...
5       комментар песк по повод диалог эстон и зелен п...
6       реакц мы решил включит став лайк за сильн рос ...
                              ...                        
1058    байден мол багат зайв і в том числ повторюв по...
1098    до реч дуч про превентивн ящерк прав тобт піту...
1113    ми зараз знаходил в схож момент на лют тод ус ...
1122    улюблен сучк такер та обєкт мастурбаці америка...
1123    смішн спостеріг за цим псевдоправ блядськ цирк...
Name: parsed_text, Length: 1135, dtype: object


TfidfVectorizer()

In [ ]:
df[df['parsed_text'].str.len() == 0]['parsed_text'] = None
df.dropna(inplace=True)
# for ls in df.parsed_text:
  # LS.append(my_vectorizer.transform(ls).toarray())
  # if len(ls) == 0:
print(my_vectorizer.transform(df.iloc[[2, 3]]['parsed_text']).toarray())
print(my_vectorizer.transform(df['parsed_text']).toarray().shape)
x = my_vectorizer.transform(df['parsed_text']).toarray()
y = df['propaganda'].apply(lambda x: 0 if x=='f' else 1).values.astype(int)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
  #   print("yes")
# print(LS)
# df['parsed_text'] = LS
print(df['parsed_text'])


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(1135, 11126)
id
1147       китайськ жіночкакішк продовж намаг в пропаганд
2       1991 росс давайт не буд ссорит мы вед брат укр...
4       мысли вслух кажд росисянин сейчас слов админис...
5       комментар песк по повод диалог эстон и зелен п...
6       реакц мы решил включит став лайк за сильн рос ...
                              ...                        
1058    байден мол багат зайв і в том числ повторюв по...
1098    до реч дуч про превентивн ящерк прав тобт піту...
1113    ми зараз знаходил в схож момент на лют тод ус ...
1122    улюблен сучк такер та обєкт мастурбаці америка...
1123    смішн спостеріг за цим псевдоправ блядськ цирк...
Name: parsed_text, Length: 1135, dtype: object


In [ ]:
# замінити текст матрицею Ps. model.fit(vectorizer(x_train).toarray(), y_train)
model = LogisticRegression()
print(df['parsed_text'], df['propaganda'])
model.fit(x_train, y_train)


id
1147       китайськ жіночкакішк продовж намаг в пропаганд
2       1991 росс давайт не буд ссорит мы вед брат укр...
4       мысли вслух кажд росисянин сейчас слов админис...
5       комментар песк по повод диалог эстон и зелен п...
6       реакц мы решил включит став лайк за сильн рос ...
                              ...                        
1058    байден мол багат зайв і в том числ повторюв по...
1098    до реч дуч про превентивн ящерк прав тобт піту...
1113    ми зараз знаходил в схож момент на лют тод ус ...
1122    улюблен сучк такер та обєкт мастурбаці америка...
1123    смішн спостеріг за цим псевдоправ блядськ цирк...
Name: parsed_text, Length: 1135, dtype: object id
1147    f
2       t
4       t
5       t
6       t
       ..
1058    f
1098    f
1113    f
1122    f
1123    f
Name: propaganda, Length: 1135, dtype: object


LogisticRegression()

In [ ]:

# print(model.predict(my_vectorizer.transform([df.iloc[532]['parsed_text']])))
# print(df.iloc[532]['parsed_text'])
model.score(x_test, y_test)

0.7917888563049853

In [ ]:
def convert_to_parsed_text(arr):
  ls = []
  for b_parsed_text in arr:
    if b_parsed_text:
      ls.append('propaganda')
    else:
      ls.append('good boy')
  return ls

In [ ]:
# print(dir(model))
x = my_vectorizer.transform(['Страшно жить в Росии', 'не знаю что и сказать', 'укроп']).toarray()
convert_to_parsed_text(model.predict(x))

['good boy', 'propaganda', 'good boy']

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AiBotProject/db.csv', index_col=['id'])
nlp_u = spacy.load('uk_core_news_sm')
new_df = df
# df[df['language_code' == 'ua']]['parsed_text'] = df[df['language_code'] == 'ua']['parsed_text'].apply(lambda x: nlp_u(x).vector_norm)
# print(df.parsed_text)
new_df['parsed_text'] =  new_df['parsed_text'].apply(lambda x: x.split(' '))
new_df['parsed_text'] = new_df['parsed_text'].apply(lambda x: np.average([nlp_u(val).vector_norm for val in x]))



In [ ]:
print(df['language_code'].value_counts())
print(df[df['language_code'] == 'ua']['propaganda'].value_counts())

rus    626
ua     511
Name: language_code, dtype: int64
f    505
t      6
Name: propaganda, dtype: int64


In [ ]:
x = new_df.parsed_text.values.reshape(-1, 1)
y = new_df['propaganda'].apply(lambda x: 0 if x=='f' else 1).values.astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [ ]:
# def double_iteration(ls2):
#   for y in ls2:
#     for x in y:
#         yield x

model = LogisticRegression()
model.fit(x_train, y_train)
# model.fit(double_iteration(x_train), y_train)
print(model.score(x_test, y_test))

0.6461988304093568


In [ ]:
# текст для перевірки моделі (на демку)
my_text = input("Введіть текст: ")
prep_text = np.average([nlp_u(val).vector_norm for val in my_text.split(' ')]).reshape(-1, 1)
print(prep_text)
print(model.predict(prep_text))

Введіть текст: щось не так
[[12.19840011]]
[0]
